In [1]:
import graphlab as gl
import graphlab.aggregate as agg
gl.set_runtime_config('GRAPHLAB_DEFAULT_NUM_PYLAMBDA_WORKERS', 32)
import os
import json

[INFO] This non-commercial license of GraphLab Create is assigned to loganwls@uw.edu and will expire on October 23, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-105076 - Server binary: /home/loganwls/.local/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1463085043.log
[INFO] GraphLab Server Version: 1.6.1


In [2]:
# Load the data retrived from the NYT API.
raw = None
keys = ['headline', 'byline', 'pub_date', 'news_desk', 'section_name', 'web_url']
types = [dict, dict, str, str, str, str]
evals = [1, 1, 0, 0, 0, 0]
for fname in os.listdir('nyt_raw'):
    path = os.path.join('nyt_raw', fname)
    if os.path.isfile(path) and fname.endswith('.json'):
        with open(path, 'rb') as f:
            j = json.load(f)['response']
        if 'docs' in j and j['docs']:
            cur_sf = gl.SFrame({k: gl.SArray([x.get(k, None) for x in j['docs']], dtype=str) for t, k in zip(types, keys)})
            for k, t, e in zip(keys, types, evals):
                if e:
                    cur_sf[k] = cur_sf[k].apply(lambda x: eval(x) if eval(x) else None, dtype=t)
                else:
                    cur_sf[k] = cur_sf[k].astype(t)
        if len(cur_sf):
            if not raw:
                raw = cur_sf
            else:
                raw = raw.append(cur_sf)
raw

byline,headline,news_desk,pub_date,section_name
"{'person':[{'organization': '', ...","{'main': ""'Fat Talk'Compels but Carries a ...",None,2013-05-27T13:15:43Z,Health
"{'person':[{'organization': '', ...",{'main': 'Really? TheClaim: Fresh Produce Has ...,None,2013-05-27T14:00:36Z,Health
"{'person': [{'firstname':'Roni', 'middlename': ...","{'main': 'The New Rulesfor Sunscreen', 'kick ...",None,2013-05-27T15:30:46Z,Health
"{'contributor': '','person': ...","{'seo': 'UnderstandingHoarding', 'main': ...",National,2013-05-28T00:00:00Z,Health
"{'contributor': '','person': [{'qualifier': ...","{'main': 'New Tools toHunt New Viruses', ...",Science,2013-05-28T00:00:00Z,Health
"{'contributor': '','person': [{'qualifier': ...","{'seo': 'Polio Reappearsin the Horn of Africa', ...",Science,2013-05-28T00:00:00Z,Health
"{'person':[{'organization': '', ...","{'main': 'The RovingRunner Rides a Bike', ...",None,2013-05-28T17:01:33Z,Health
"{'person':[{'organization': '', ...","{'main': 'Is It Better toWalk or Run?', 'kicker': ...",None,2013-05-29T00:01:50Z,Health
"{'person':[{'organization': '', ...","{'main': 'What Not to Sayat the Bedside', ...",None,2013-05-29T13:05:04Z,Health
"{'person':[{'organization': '', ...",{'main': 'Winning theMRSA Battle in ...,None,2013-05-29T17:47:59Z,Health


In [3]:
# Extract the author name.
raw['author'] = raw['byline'].apply(lambda x: x['original'].replace('By ', ''))
raw

byline,headline,news_desk,pub_date,section_name
"{'person':[{'organization': '', ...","{'main': ""'Fat Talk'Compels but Carries a ...",None,2013-05-27T13:15:43Z,Health
"{'person':[{'organization': '', ...",{'main': 'Really? TheClaim: Fresh Produce Has ...,None,2013-05-27T14:00:36Z,Health
"{'person': [{'firstname':'Roni', 'middlename': ...","{'main': 'The New Rulesfor Sunscreen', 'kick ...",None,2013-05-27T15:30:46Z,Health
"{'contributor': '','person': ...","{'seo': 'UnderstandingHoarding', 'main': ...",National,2013-05-28T00:00:00Z,Health
"{'contributor': '','person': [{'qualifier': ...","{'main': 'New Tools toHunt New Viruses', ...",Science,2013-05-28T00:00:00Z,Health
"{'contributor': '','person': [{'qualifier': ...","{'seo': 'Polio Reappearsin the Horn of Africa', ...",Science,2013-05-28T00:00:00Z,Health
"{'person':[{'organization': '', ...","{'main': 'The RovingRunner Rides a Bike', ...",None,2013-05-28T17:01:33Z,Health
"{'person':[{'organization': '', ...","{'main': 'Is It Better toWalk or Run?', 'kicker': ...",None,2013-05-29T00:01:50Z,Health
"{'person':[{'organization': '', ...","{'main': 'What Not to Sayat the Bedside', ...",None,2013-05-29T13:05:04Z,Health
"{'person':[{'organization': '', ...",{'main': 'Winning theMRSA Battle in ...,None,2013-05-29T17:47:59Z,Health


In [4]:
# Extract the main headline
raw.rename({'headline':'raw_headline'})
raw['headline'] = raw['raw_headline'].apply(lambda x: x['main'])
raw

byline,raw_headline,news_desk,pub_date,section_name
"{'person':[{'organization': '', ...","{'main': ""'Fat Talk'Compels but Carries a ...",None,2013-05-27T13:15:43Z,Health
"{'person':[{'organization': '', ...",{'main': 'Really? TheClaim: Fresh Produce Has ...,None,2013-05-27T14:00:36Z,Health
"{'person': [{'firstname':'Roni', 'middlename': ...","{'main': 'The New Rulesfor Sunscreen', 'kick ...",None,2013-05-27T15:30:46Z,Health
"{'contributor': '','person': ...","{'seo': 'UnderstandingHoarding', 'main': ...",National,2013-05-28T00:00:00Z,Health
"{'contributor': '','person': [{'qualifier': ...","{'main': 'New Tools toHunt New Viruses', ...",Science,2013-05-28T00:00:00Z,Health
"{'contributor': '','person': [{'qualifier': ...","{'seo': 'Polio Reappearsin the Horn of Africa', ...",Science,2013-05-28T00:00:00Z,Health
"{'person':[{'organization': '', ...","{'main': 'The RovingRunner Rides a Bike', ...",None,2013-05-28T17:01:33Z,Health
"{'person':[{'organization': '', ...","{'main': 'Is It Better toWalk or Run?', 'kicker': ...",None,2013-05-29T00:01:50Z,Health
"{'person':[{'organization': '', ...","{'main': 'What Not to Sayat the Bedside', ...",None,2013-05-29T13:05:04Z,Health
"{'person':[{'organization': '', ...",{'main': 'Winning theMRSA Battle in ...,None,2013-05-29T17:47:59Z,Health


In [5]:
# Load the scraped NYT data.
scraped = None
types = [list, str, list, list]
evals = [1, 0, 1, 1]
keys = ['link_ids', 'body', 'paper_links', 'links']
for fname in os.listdir('nyt_processed'):
    path = os.path.join('nyt_processed', fname)
    if os.path.isfile(path) and fname.endswith('.json'):
        with open(path, 'rb') as f:
            j = json.load(f)
        if 'docs' in j and j['docs']:
            cur_sf = gl.SFrame({k: gl.SArray([x.get(k, None) for x in j['docs']], dtype=str) for t, k in zip(types, keys)})
            for k, t, e in zip(keys, types, evals):
                if e:
                    cur_sf[k] = cur_sf[k].apply(lambda x: eval(x) if eval(x) else None, dtype=t)
                else:
                    cur_sf[k] = cur_sf[k].astype(t)

        if len(cur_sf):
            if not scraped:
                scraped = cur_sf
            else:
                scraped = scraped.append(cur_sf)
scraped

In [6]:
# Combine the two into one SFrame.
nyt = raw.add_row_number('nyt_id').join(scraped.add_row_number('nyt_id'), on='nyt_id', how='outer')
nyt

In [7]:
# Load the data from the followed links

# Merges DOIs retrieved by different methods
def combine(row):
    if row['DOI'] and row['ID_TYPE']:
        return row['DOI'] + [row['ID_TYPE']]
    elif row['DOI'] and not row['ID_TYPE']:
        return row['DOI']
    elif row['ID_TYPE'] and not row['DOI']:
        return [row['ID_TYPE']]
    else:
        return None

d = []
for fname in os.listdir('archive_data/nyt_processed_urls'):
    path = os.path.join('archive_data/nyt_processed_urls', fname)
    if os.path.isfile(path) and fname.endswith('.json'):
        with open(path, 'rb') as f:
            j = json.load(f)
        if j:
            d.append(j)
            
links = gl.SFrame(d).unpack('X1', column_name_prefix='')
links.rename({'0':'link_id', '1':'link_url', '2':'response'})
links = links.unpack('response', column_name_prefix='')

In [8]:
# Reformat the link data as a dict.
links['ids'] = links['ids'].apply(lambda x: {i[0]:i[1] for i in x})
links

link_id,link_url,authors,error,ids,raw_parse
108,http://www.thelancet.com/journals/laninf/artic ...,[],None,{'DOI':['10.1016/S1473']} ...,
366,http://pediatrics.aappublications.org/cgi/doi/ ...,[],None,{'DOI': []},
1694,http://journals.lww.com/academicmedicine/Fullt ...,[],None,{'DOI': ['10.1097/ACM.0b013e3182864299']} ...,
875,http://www.pnas.org/content/early/2013/11/27/1 ...,[],None,{'DOI': ['10.1073/pnas.1312857110/']} ...,
1304,http://archinte.jamanetwork.com/article.aspx?d ...,[],None,"{'DOI': ['10.1001/jamainternmed.2013.2912.', ' ...",
1575,http://www.ncbi.nlm.nih.gov/pubmed/21284037 ...,[],None,"{'DOI':['10.1002/mds.23449.'], ...",
1795,http://www.va.gov/vaforms/va/pdf/VA9.pdf ...,[],None,{},"<?xml version=""1.0""encoding=""UTF-8""?>\n< ..."
784,http://sociology.sas.upenn.edu/annette_lareau ...,[],None,{'DOI': []},
2012,http://www.sciencemag.org/content/323/5918/1226 ...,[],None,{'DOI': ['10.1126/science.1169144']} ...,
1059,http://gas.sagepub.com/content/early/2012/10/2 ...,[],None,{'DOI': []},


In [9]:
# Unpack the link data to columns
links = links.unpack('ids', column_name_prefix='')
links

link_id,link_url,authors,error,raw_parse,titles
108,http://www.thelancet.com/journals/laninf/artic ...,[],None,,[]
366,http://pediatrics.aappublications.org/cgi/doi/ ...,[],None,,[]
1694,http://journals.lww.com/academicmedicine/Fullt ...,[],None,,[]
875,http://www.pnas.org/content/early/2013/11/27/1 ...,[],None,,[]
1304,http://archinte.jamanetwork.com/article.aspx?d ...,[],None,,[]
1575,http://www.ncbi.nlm.nih.gov/pubmed/21284037 ...,[],None,,[]
1795,http://www.va.gov/vaforms/va/pdf/VA9.pdf ...,[],None,"<?xml version=""1.0""encoding=""UTF-8""?>\n< ...","[APPEAL TO BOARD OFVETERANS' APPEALS, AP ..."
784,http://sociology.sas.upenn.edu/annette_lareau ...,[],None,,[]
2012,http://www.sciencemag.org/content/323/5918/1226 ...,[],None,,[]
1059,http://gas.sagepub.com/content/early/2012/10/2 ...,[],None,,[]


In [10]:
# Combine the multiple DOI columns and reformat them.
links['DOI'] = links.apply(combine)
del links['ID_TYPE']
links = links.stack('DOI', 'DOI').stack('PUBMED','PUBMED')
links

link_id,link_url,authors,error,raw_parse,titles,DOI,PUBMED
108,http://www.thelancet.com/journals/laninf/artic ...,[],None,,[],10.1016/S1473,None
366,http://pediatrics.aappublications.org/cgi/doi/ ...,[],None,,[],None,None
1694,http://journals.lww.com/academicmedicine/Fullt ...,[],None,,[],10.1097/ACM.0b013e3182864299 ...,None
875,http://www.pnas.org/content/early/2013/11/27/1 ...,[],None,,[],10.1073/pnas.1312857110/,None
1304,http://archinte.jamanetwork.com/article.aspx?d ...,[],None,,[],10.1001/jamainternmed.2013.2912. ...,None
1304,http://archinte.jamanetwork.com/article.aspx?d ...,[],None,,[],10.1001/jamainternmed.2014.3901. ...,None
1304,http://archinte.jamanetwork.com/article.aspx?d ...,[],None,,[],10.1001/jamainternmed.2015.4585. ...,None
1304,http://archinte.jamanetwork.com/article.aspx?d ...,[],None,,[],10.1001/jamainternmed.2013.2908. ...,None
1304,http://archinte.jamanetwork.com/article.aspx?d ...,[],None,,[],10.1001/jamainternmed.2013.9257. ...,None
1304,http://archinte.jamanetwork.com/article.aspx?d ...,[],None,,[],10.1001/jamainternmed.2015.4594. ...,None


In [20]:
# Attatch the DOIs to the NYT articles using the link_ids
nyt_to_links = nyt[['nyt_id','link_ids']].stack('link_ids', 'link_id').join(links, on='link_id', how='left')
nyt_to_DOI = nyt_to_links.groupby('nyt_id', {'DOI':agg.CONCAT('DOI')})
nyt = nyt.join(nyt_to_DOI, on='nyt_id', how='outer')
nyt

nyt_id,byline,raw_headline,news_desk,pub_date,section_name
0,"{'person':[{'organization': '', ...","{'main': ""'Fat Talk'Compels but Carries a ...",None,2013-05-27T13:15:43Z,Health
1,"{'person':[{'organization': '', ...",{'main': 'Really? TheClaim: Fresh Produce Has ...,None,2013-05-27T14:00:36Z,Health
2,"{'person': [{'firstname':'Roni', 'middlename': ...","{'main': 'The New Rulesfor Sunscreen', 'kick ...",None,2013-05-27T15:30:46Z,Health
3,"{'contributor': '','person': ...","{'seo': 'UnderstandingHoarding', 'main': ...",National,2013-05-28T00:00:00Z,Health
4,"{'contributor': '','person': [{'qualifier': ...","{'main': 'New Tools toHunt New Viruses', ...",Science,2013-05-28T00:00:00Z,Health
5,"{'contributor': '','person': [{'qualifier': ...","{'seo': 'Polio Reappearsin the Horn of Africa', ...",Science,2013-05-28T00:00:00Z,Health
6,"{'person':[{'organization': '', ...","{'main': 'The RovingRunner Rides a Bike', ...",None,2013-05-28T17:01:33Z,Health
7,"{'person':[{'organization': '', ...","{'main': 'Is It Better toWalk or Run?', 'kicker': ...",None,2013-05-29T00:01:50Z,Health
8,"{'person':[{'organization': '', ...","{'main': 'What Not to Sayat the Bedside', ...",None,2013-05-29T13:05:04Z,Health
9,"{'person':[{'organization': '', ...",{'main': 'Winning theMRSA Battle in ...,None,2013-05-29T17:47:59Z,Health


In [21]:
# Save the data
nyt.save('new_clean/nyt.sf')

In [28]:
# Load the DOI metadata from the lookup service
metadata = None
keys = ['DOI', 'info']
types = [str, dict]
evals = [0, 1]

d = []
for p in ['archive_data/metadata', 'archive_data/metadata2']:
    for fname in os.listdir(p):
        path = os.path.join(p, fname)
        if os.path.isfile(path) and fname.endswith('.json'):
            with open(path, 'rb') as f:
                j = json.load(f)
                d.append(j)
#             cur_sf = gl.SFrame({k: gl.SArray([j.get(k, None)], dtype=str) for t, k in zip(types, keys)})
#             for k, t, e in zip(keys, types, evals):
#                 if e:
#                     cur_sf[k] = cur_sf[k].apply(lambda x: eval(x) if eval(x) else None, dtype=t)
#                 else:
#                     cur_sf[k] = cur_sf[k].astype(t)
#             if len(cur_sf):
#                 if not metadata:
#                     metadata = cur_sf
#                 else:
#                     metadata = metadata.append(cur_sf)
metadata = gl.SFrame(d)
metadata

X1
"{'info': {'error': None,'publication_info': ..."
"{'info': {'error': None,'publication_info': ..."
"{'info': {'error': None,'publication_info': [], ..."
"{'info': {'error': None,'publication_info': ..."
"{'info': {'error': '404','publication_info': [], ..."
"{'info': {'error': None,'publication_info': ..."
"{'info': {'error': None,'publication_info': ..."
"{'info': {'error': '404','publication_info': [], ..."
"{'info': {'error': None,'publication_info': ..."
"{'info': {'error': None,'publication_info': ..."


In [93]:
#Unpack the data and re-format it.
unpacked = metadata.unpack('X1', column_name_prefix='').unpack('info', column_name_prefix='')
unpacked['paper_info'] = unpacked['paper_info'].apply(lambda x: x[0] if x else None)
unpacked['publication_info'] = unpacked['publication_info'].apply(lambda x: x[0] if x else None)
unpacked = unpacked.unpack('paper_info', column_name_prefix='').remove_columns(['pageEnd',
                                                                                'endingPage',
                                                                                'pageStart',
                                                                                'startingPage',
                                                                                'isPartOf',
                                                                                'owl#sameAs',
                                                                                'doi',
                                                                                'dtype',
                                                                                'volume',
                                                                                'creator',
                                                                                'other'])
unpacked['author'] = unpacked['authors'].apply(lambda x: [i.get('name', None)[0] for i in x])
unpacked['author_url'] = unpacked['authors'].apply(lambda x: [i.get('url', None) for i in x])
unpacked = unpacked.unpack('publication_info', column_name_prefix='').remove_columns(['owl#sameAs.1',
                                                                                     'url',
                                                                                      'url.1',
                                                                                     'dtype.1',
                                                                                     'authors'])
unpacked.rename({'title.1': 'publication_name', 'issn.1':'issn', 'identifier':'api_returned_doi', 'date':'pub_date'})

for c in ['publisher', 'title', 'pub_date', 'publication_name', 'author_url']:
    test = unpacked.stack(c, c)
    if test.num_rows() == unpacked.num_rows() or c == 'title':
        unpacked[c] = unpacked[c].apply(lambda x: x[0] if x else None)

# Drop all of the requests which errored out
errors, unpacked = unpacked.dropna_split('error')
del unpacked['error']
unpacked

DOI,pub_date,api_returned_doi,publisher
10.1136/bmj.f1140,2013-3-18,[10.1136/bmj.f1140],BMJ
10.1162/089892998562997,1998-9,[10.1162/089892998562997],MIT Press - Journals
10.1371/journal.pone.0064172.g001 ...,None,[10.1371/journal.pone.0064172.g001] ...,Public Library of Science(PLoS) ...
10.1016/j.cmet.2011.03.013 ...,2011-4,[10.1016/j.cmet.2011.03.013] ...,Elsevier BV
10.1542/peds.2012-2368,2013-4-8,[10.1542/peds.2012-2368],American Academy ofPediatrics (AAP) ...
10.1056/nejm199302253280804 ...,1993-2-25,[10.1056/nejm199302253280804] ...,New England Journal ofMedicine (NEJM/MMS) ...
10.1136/bmj.b92,2009-2-16,[10.1136/bmj.b92],BMJ
10.1056/NEJMra020995,2003-10-2,[10.1056/nejmra020995],New England Journal ofMedicine (NEJM/MMS) ...
10.1371/journal.pbio.1001603.g001 ...,None,[10.1371/journal.pbio.1001603.g001] ...,Public Library of Science(PLoS) ...
10.1136/bjsm.37.3.239,2003-6-1,[10.1136/bjsm.37.3.239],BMJ


In [94]:
print len(unpacked['DOI'].unique())
print unpacked.unique().groupby('DOI',{'count':agg.COUNT()}).sort('count', False)
unpacked.unique().filter_by(['10.1371/journal.pone.0063416'], 'DOI')

# The formatting of the dates is what causes this duplication so
# we will fix it like so:
dates = unpacked.groupby('DOI',{'pub_date':agg.SELECT_ONE('pub_date')})
del unpacked['pub_date']
unpacked = unpacked.unique().join(dates, on='DOI', how='left')
unpacked

2106
+-------------------------------+-------+
|              DOI              | count |
+-------------------------------+-------+
|     10.2471/BLT.13.119230     |   2   |
|  10.1371/journal.pone.0063416 |   2   |
| 10.1001/jamainternmed.2013... |   2   |
|  10.1371/journal.pone.0063277 |   2   |
|  10.1371/journal.pone.0066949 |   2   |
|      10.1289/ehp.1205575      |   2   |
|  10.1371/journal.pmed.1001430 |   2   |
|  10.1371/journal.pone.0060676 |   2   |
|  10.1371/journal.pone.0066658 |   2   |
|  10.1371/journal.pgen.1003553 |   2   |
+-------------------------------+-------+
[2106 rows x 2 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


DOI,api_returned_doi,author,author_url
10.1016/S0140-6736(13)61647-5 ...,[10.1016/s0140-6736(13)61647-5] ...,"[Andrew Grey, Mark JBolland, Ian R Reid] ...",[http://id.crossref.org/contributor/andrew-grey- ...
10.1001/jama.2013.5892,[10.1001/jama.2013.5892],"[Patty J. Nelemans, RobA. de Bie, J. Bart St ...",[http://id.crossref.org/contributor/patty-j ...
10.1056/nejmsa053935,[10.1056/nejmsa053935],"[Earl S. Ford, Julia A.Critchley, Thomas E. ...",[http://id.crossref.org/contributor/earl-s-ford- ...
10.1016/j.tics.2004.11.007 ...,[10.1016/j.tics.2004.11.007] ...,"[Susan E. Rogers, DanielJ. Levitin] ...",[http://id.crossref.org/contributor/susan-e ...
10.1002/oby.20384,[10.1002/oby.20384],"[Debra L. Roter, KimberlyA. Gudzune, Mary ...",[http://id.crossref.org/contributor/debra-l-ro ...
10.1056/NEJMp1215606,[10.1056/nejmp1215606],"[Sean Palfrey, Judith S.Palfrey] ...",[http://id.crossref.org/contributor/sean-palfrey- ...
10.1056/nejmsa042657,[10.1056/nejmsa042657],"[Christopher P.Neukermans, Gillian D. ...",[http://id.crossref.org/contributor/christopher-p ...
10.1161/01.cir.67.1.24,[10.1161/01.cir.67.1.24],"[T. D. Noakes, L. H.Opie, L. Higginson] ...",[http://id.crossref.org/contributor/t-d-noakes- ...
10.1177/0363546511433279,[10.1177/0363546511433279] ...,"[J. E. Shealy, S. Kim, C.F. Ettlinger, R. J. ...",[http://id.crossref.org/contributor/j-e-shealy- ...
10.3357/asem.2538.2009,[10.3357/asem.2538.2009],"[Sebastien Villard, YawenYu, Thomas A. Stoffre ...",[http://id.crossref.org/contributor/sebastien- ...


In [95]:
eigen = gl.SFrame('clean/eigenfactor.tsv')
# Delete the database ID for the papers
del eigen['Paper_ID']
eigen.rename({'Paper_Document_Object_Identifier_(DOI)':'DOI', 'EF':'eigenfactor'})
eigen = eigen.unique()

# Not sure why some have multiple eigenfactors, for now we will use the mean.
eigen = eigen.groupby('DOI', {'eigenfactor':agg.MEAN('eigenfactor')})
unpacked = unpacked.join(eigen, on='DOI', how='left')

PROGRESS: Finished parsing file /home/loganwls/pane/clean/eigenfactor.tsv
PROGRESS: Parsing completed. Parsed 100 lines in 0.039406 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,str,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /home/loganwls/pane/clean/eigenfactor.tsv
PROGRESS: Parsing completed. Parsed 1576 lines in 0.036402 secs.


In [96]:
unpacked.save('new_clean/academic_articles.sf')

In [98]:
nyt

nyt_id,byline,raw_headline,news_desk,pub_date,section_name
0,"{'person':[{'organization': '', ...","{'main': ""'Fat Talk'Compels but Carries a ...",None,2013-05-27T13:15:43Z,Health
1,"{'person':[{'organization': '', ...",{'main': 'Really? TheClaim: Fresh Produce Has ...,None,2013-05-27T14:00:36Z,Health
2,"{'person': [{'firstname':'Roni', 'middlename': ...","{'main': 'The New Rulesfor Sunscreen', 'kick ...",None,2013-05-27T15:30:46Z,Health
3,"{'contributor': '','person': ...","{'seo': 'UnderstandingHoarding', 'main': ...",National,2013-05-28T00:00:00Z,Health
4,"{'contributor': '','person': [{'qualifier': ...","{'main': 'New Tools toHunt New Viruses', ...",Science,2013-05-28T00:00:00Z,Health
5,"{'contributor': '','person': [{'qualifier': ...","{'seo': 'Polio Reappearsin the Horn of Africa', ...",Science,2013-05-28T00:00:00Z,Health
6,"{'person':[{'organization': '', ...","{'main': 'The RovingRunner Rides a Bike', ...",None,2013-05-28T17:01:33Z,Health
7,"{'person':[{'organization': '', ...","{'main': 'Is It Better toWalk or Run?', 'kicker': ...",None,2013-05-29T00:01:50Z,Health
8,"{'person':[{'organization': '', ...","{'main': 'What Not to Sayat the Bedside', ...",None,2013-05-29T13:05:04Z,Health
9,"{'person':[{'organization': '', ...",{'main': 'Winning theMRSA Battle in ...,None,2013-05-29T17:47:59Z,Health


In [108]:
# Some DOIs were malformed in the original data so we need to ensure the proper DOIs are 

import re
fixer = re.compile(r'(10[.][0-9]{4,}(?:[.][0-9]+)*/(?:(?!["&\'<>])\S)+[0-9])', re.IGNORECASE)

def fix(x):
    if x:
        tmp = re.search(fixer, x)
        if tmp:
            return tmp.groups()[0]
    return None

def final_doi(x):
    if x['academic_id']:
        return x['DOI']
    elif x['academic_id.1']:
        return x['fixed']
    else:
        return None
        

nyt_to_doi = nyt[['nyt_id', 'DOI']].stack('DOI', 'DOI')
nyt_to_doi['fixed'] = nyt_to_doi['DOI'].apply(fix)
nyt_to_doi

tmp = unpacked.add_row_number('academic_id')[['DOI', 'academic_id']]
nyt_to_doi = nyt_to_doi.join(tmp, on='DOI', how='left').join(tmp, on={'fixed':'DOI'}, how='left')
nyt_to_doi['final_doi'] = nyt_to_doi.apply(final_doi)
nyt_to_doi.dropna('final_doi')

nyt_to_doi.remove_columns(['DOI', 'fixed', 'academic_id', 'academic_id.1'])
nyt_to_doi.rename({'final_doi':'DOI'})

nyt_id,byline,raw_headline,news_desk,pub_date,section_name
0,"{'person':[{'organization': '', ...","{'main': ""'Fat Talk'Compels but Carries a ...",None,2013-05-27T13:15:43Z,Health
1,"{'person':[{'organization': '', ...",{'main': 'Really? TheClaim: Fresh Produce Has ...,None,2013-05-27T14:00:36Z,Health
2,"{'person': [{'firstname':'Roni', 'middlename': ...","{'main': 'The New Rulesfor Sunscreen', 'kick ...",None,2013-05-27T15:30:46Z,Health
3,"{'contributor': '','person': ...","{'seo': 'UnderstandingHoarding', 'main': ...",National,2013-05-28T00:00:00Z,Health
4,"{'contributor': '','person': [{'qualifier': ...","{'main': 'New Tools toHunt New Viruses', ...",Science,2013-05-28T00:00:00Z,Health
5,"{'contributor': '','person': [{'qualifier': ...","{'seo': 'Polio Reappearsin the Horn of Africa', ...",Science,2013-05-28T00:00:00Z,Health
6,"{'person':[{'organization': '', ...","{'main': 'The RovingRunner Rides a Bike', ...",None,2013-05-28T17:01:33Z,Health
7,"{'person':[{'organization': '', ...","{'main': 'Is It Better toWalk or Run?', 'kicker': ...",None,2013-05-29T00:01:50Z,Health
8,"{'person':[{'organization': '', ...","{'main': 'What Not to Sayat the Bedside', ...",None,2013-05-29T13:05:04Z,Health
9,"{'person':[{'organization': '', ...",{'main': 'Winning theMRSA Battle in ...,None,2013-05-29T17:47:59Z,Health


In [113]:
# Save this as our article graph edges
nyt_to_doi.save('new_clean/article_network/edges.csv')

In [109]:
# Join the corrected DOIs back into the NYT data and save.
nyt = nyt.join(nyt_to_doi.unstack('DOI', 'DOI'), on='nyt_id', how='left')
del nyt['DOI']
nyt.rename({'DOI.1':'DOI'})
nyt.save('new_clean/nyt.sf')

In [116]:
unpacked.save('new_clean/academic_full.sf')

In [117]:
aca = unpacked.filter_by(nyt_to_doi['DOI'].unique(), 'DOI')
aca

DOI,api_returned_doi,author,author_url
10.1001/jama.2013.5892,[10.1001/jama.2013.5892],"[Patty J. Nelemans, RobA. de Bie, J. Bart St ...",[http://id.crossref.org/contributor/patty-j ...
10.1056/nejmsa053935,[10.1056/nejmsa053935],"[Earl S. Ford, Julia A.Critchley, Thomas E. ...",[http://id.crossref.org/contributor/earl-s-ford- ...
10.1016/j.tics.2004.11.007 ...,[10.1016/j.tics.2004.11.007] ...,"[Susan E. Rogers, DanielJ. Levitin] ...",[http://id.crossref.org/contributor/susan-e ...
10.1002/oby.20384,[10.1002/oby.20384],"[Debra L. Roter, KimberlyA. Gudzune, Mary ...",[http://id.crossref.org/contributor/debra-l-ro ...
10.1056/NEJMp1215606,[10.1056/nejmp1215606],"[Sean Palfrey, Judith S.Palfrey] ...",[http://id.crossref.org/contributor/sean-palfrey- ...
10.1056/nejmsa042657,[10.1056/nejmsa042657],"[Christopher P.Neukermans, Gillian D. ...",[http://id.crossref.org/contributor/christopher-p ...
10.1161/01.cir.67.1.24,[10.1161/01.cir.67.1.24],"[T. D. Noakes, L. H.Opie, L. Higginson] ...",[http://id.crossref.org/contributor/t-d-noakes- ...
10.1177/0363546511433279,[10.1177/0363546511433279] ...,"[J. E. Shealy, S. Kim, C.F. Ettlinger, R. J. ...",[http://id.crossref.org/contributor/j-e-shealy- ...
10.3357/asem.2538.2009,[10.3357/asem.2538.2009],"[Sebastien Villard, YawenYu, Thomas A. Stoffre ...",[http://id.crossref.org/contributor/sebastien- ...
10.1001/jama.2010.303,[10.1001/jama.2010.303],[Brenda R. Hemmelgarn],[http://id.crossref.org/contributor/brenda-r ...


In [128]:
# Add the open_access marker
oa = gl.SFrame('name_match.csv')
oa['true_match'] = oa.apply(lambda x: True if bool(x['is_match']) and bool(['Tin']) else None)
open_access_names = oa.dropna('true_match')['publication_title']
oa_marker = gl.SFrame({'publication_name':open_access_names, 'open_access':gl.SArray.from_const(True, len(open_access_names))})
aca = aca.join(oa_marker, on='publication_name', how='left')

PROGRESS: Finished parsing file /home/loganwls/pane/name_match.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.040742 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[array,str,str,int,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /home/loganwls/pane/name_match.csv
PROGRESS: Parsing completed. Parsed 615 lines in 0.034123 secs.


In [129]:
aca.save('new_clean/academic_articles.sf')

In [120]:
nyt

nyt_id,byline,raw_headline,news_desk,pub_date,section_name
0,"{'person':[{'organization': '', ...","{'main': ""'Fat Talk'Compels but Carries a ...",None,2013-05-27T13:15:43Z,Health
1,"{'person':[{'organization': '', ...",{'main': 'Really? TheClaim: Fresh Produce Has ...,None,2013-05-27T14:00:36Z,Health
2,"{'person': [{'firstname':'Roni', 'middlename': ...","{'main': 'The New Rulesfor Sunscreen', 'kick ...",None,2013-05-27T15:30:46Z,Health
3,"{'contributor': '','person': ...","{'seo': 'UnderstandingHoarding', 'main': ...",National,2013-05-28T00:00:00Z,Health
4,"{'contributor': '','person': [{'qualifier': ...","{'main': 'New Tools toHunt New Viruses', ...",Science,2013-05-28T00:00:00Z,Health
5,"{'contributor': '','person': [{'qualifier': ...","{'seo': 'Polio Reappearsin the Horn of Africa', ...",Science,2013-05-28T00:00:00Z,Health
6,"{'person':[{'organization': '', ...","{'main': 'The RovingRunner Rides a Bike', ...",None,2013-05-28T17:01:33Z,Health
7,"{'person':[{'organization': '', ...","{'main': 'Is It Better toWalk or Run?', 'kicker': ...",None,2013-05-29T00:01:50Z,Health
8,"{'person':[{'organization': '', ...","{'main': 'What Not to Sayat the Bedside', ...",None,2013-05-29T13:05:04Z,Health
9,"{'person':[{'organization': '', ...",{'main': 'Winning theMRSA Battle in ...,None,2013-05-29T17:47:59Z,Health


In [130]:
aca

DOI,api_returned_doi,author,author_url
10.1001/jama.2013.5892,[10.1001/jama.2013.5892],"[Patty J. Nelemans, RobA. de Bie, J. Bart St ...",[http://id.crossref.org/contributor/patty-j ...
10.1056/nejmsa053935,[10.1056/nejmsa053935],"[Earl S. Ford, Julia A.Critchley, Thomas E. ...",[http://id.crossref.org/contributor/earl-s-ford- ...
10.1016/j.tics.2004.11.007 ...,[10.1016/j.tics.2004.11.007] ...,"[Susan E. Rogers, DanielJ. Levitin] ...",[http://id.crossref.org/contributor/susan-e ...
10.1002/oby.20384,[10.1002/oby.20384],"[Debra L. Roter, KimberlyA. Gudzune, Mary ...",[http://id.crossref.org/contributor/debra-l-ro ...
10.1056/NEJMp1215606,[10.1056/nejmp1215606],"[Sean Palfrey, Judith S.Palfrey] ...",[http://id.crossref.org/contributor/sean-palfrey- ...
10.1056/nejmsa042657,[10.1056/nejmsa042657],"[Christopher P.Neukermans, Gillian D. ...",[http://id.crossref.org/contributor/christopher-p ...
10.1161/01.cir.67.1.24,[10.1161/01.cir.67.1.24],"[T. D. Noakes, L. H.Opie, L. Higginson] ...",[http://id.crossref.org/contributor/t-d-noakes- ...
10.1177/0363546511433279,[10.1177/0363546511433279] ...,"[J. E. Shealy, S. Kim, C.F. Ettlinger, R. J. ...",[http://id.crossref.org/contributor/j-e-shealy- ...
10.3357/asem.2538.2009,[10.3357/asem.2538.2009],"[Sebastien Villard, YawenYu, Thomas A. Stoffre ...",[http://id.crossref.org/contributor/sebastien- ...
10.1001/jama.2010.303,[10.1001/jama.2010.303],[Brenda R. Hemmelgarn],[http://id.crossref.org/contributor/brenda-r ...


In [131]:
from datetime import datetime
def convert_to_datetime(x):
    #     y,m,d
    dt = [1,1,1]
    s = map(int, x.split('-'))
    for i in range(3):
        if i < len(s):
            dt[i] = s[i]
    return datetime(year=dt[0], month=dt[1], day=dt[2])

# Convert the date column for the academic papers to datetime
aca['pub_date'] = aca['pub_date'].apply(convert_to_datetime)

In [132]:
aca

DOI,api_returned_doi,author,author_url
10.1001/jama.2013.5892,[10.1001/jama.2013.5892],"[Patty J. Nelemans, RobA. de Bie, J. Bart St ...",[http://id.crossref.org/contributor/patty-j ...
10.1056/nejmsa053935,[10.1056/nejmsa053935],"[Earl S. Ford, Julia A.Critchley, Thomas E. ...",[http://id.crossref.org/contributor/earl-s-ford- ...
10.1016/j.tics.2004.11.007 ...,[10.1016/j.tics.2004.11.007] ...,"[Susan E. Rogers, DanielJ. Levitin] ...",[http://id.crossref.org/contributor/susan-e ...
10.1002/oby.20384,[10.1002/oby.20384],"[Debra L. Roter, KimberlyA. Gudzune, Mary ...",[http://id.crossref.org/contributor/debra-l-ro ...
10.1056/NEJMp1215606,[10.1056/nejmp1215606],"[Sean Palfrey, Judith S.Palfrey] ...",[http://id.crossref.org/contributor/sean-palfrey- ...
10.1056/nejmsa042657,[10.1056/nejmsa042657],"[Christopher P.Neukermans, Gillian D. ...",[http://id.crossref.org/contributor/christopher-p ...
10.1161/01.cir.67.1.24,[10.1161/01.cir.67.1.24],"[T. D. Noakes, L. H.Opie, L. Higginson] ...",[http://id.crossref.org/contributor/t-d-noakes- ...
10.1177/0363546511433279,[10.1177/0363546511433279] ...,"[J. E. Shealy, S. Kim, C.F. Ettlinger, R. J. ...",[http://id.crossref.org/contributor/j-e-shealy- ...
10.3357/asem.2538.2009,[10.3357/asem.2538.2009],"[Sebastien Villard, YawenYu, Thomas A. Stoffre ...",[http://id.crossref.org/contributor/sebastien- ...
10.1001/jama.2010.303,[10.1001/jama.2010.303],[Brenda R. Hemmelgarn],[http://id.crossref.org/contributor/brenda-r ...


In [133]:
aca.save('new_clean/academic_articles.sf')

In [136]:
# Convert nyt pub_dates to datetimes.
nyt['pub_date'] = nyt['pub_date'].str_to_datetime()

In [137]:
nyt

nyt_id,byline,raw_headline,news_desk,pub_date,section_name
0,"{'person':[{'organization': '', ...","{'main': ""'Fat Talk'Compels but Carries a ...",None,2013-05-27 13:15:43+00:00,Health
1,"{'person':[{'organization': '', ...",{'main': 'Really? TheClaim: Fresh Produce Has ...,None,2013-05-27 14:00:36+00:00,Health
2,"{'person': [{'firstname':'Roni', 'middlename': ...","{'main': 'The New Rulesfor Sunscreen', 'kick ...",None,2013-05-27 15:30:46+00:00,Health
3,"{'contributor': '','person': ...","{'seo': 'UnderstandingHoarding', 'main': ...",National,2013-05-28 00:00:00+00:00,Health
4,"{'contributor': '','person': [{'qualifier': ...","{'main': 'New Tools toHunt New Viruses', ...",Science,2013-05-28 00:00:00+00:00,Health
5,"{'contributor': '','person': [{'qualifier': ...","{'seo': 'Polio Reappearsin the Horn of Africa', ...",Science,2013-05-28 00:00:00+00:00,Health
6,"{'person':[{'organization': '', ...","{'main': 'The RovingRunner Rides a Bike', ...",None,2013-05-28 17:01:33+00:00,Health
7,"{'person':[{'organization': '', ...","{'main': 'Is It Better toWalk or Run?', 'kicker': ...",None,2013-05-29 00:01:50+00:00,Health
8,"{'person':[{'organization': '', ...","{'main': 'What Not to Sayat the Bedside', ...",None,2013-05-29 13:05:04+00:00,Health
9,"{'person':[{'organization': '', ...",{'main': 'Winning theMRSA Battle in ...,None,2013-05-29 17:47:59+00:00,Health


In [138]:
nyt.save('new_clean/nyt.sf')